Imports ...

In [1]:
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
import pickle
from typing import Dict, List, Tuple, Callable

Read in data ...

In [2]:
demographics = pickle.load(open('data/demographics.pkl', 'rb'))

comb_counts_feb = pickle.load(open('data/comb_counts_feb.pkl', 'rb'))
comb_counts_apr = pickle.load(open('data/comb_counts_apr.pkl', 'rb'))

trip_counts_feb = pickle.load(open('data/trip_counts_feb.pkl', 'rb'))
trip_counts_apr = pickle.load(open('data/trip_counts_apr.pkl', 'rb'))

In [3]:
# store all CBGs in order
ordered_cbgs = sorted(demographics.keys())

In [4]:
# percentage change of number of trips: April / February
trip_count_change = {}

for cbg in ordered_cbgs:

    if cbg in trip_counts_feb and cbg in trip_counts_apr:
        change = trip_counts_apr[cbg] / trip_counts_feb[cbg]

    else:
        change = 1
        
    trip_count_change[cbg] = change

Create adjacency list etc. between CBGs...

In [5]:
def create_adjacency_list(comb_counts: Dict[Tuple[str, str], int], 
                          trip_counts: Dict[str, int]) -> Dict[str, List[float]]:
    """
    Create an adjacency list in the form of a dictionary. 
    The keys are the CGBs and the values are ordered lists of transition probabilities to other CBGs.
    The probabilities are in the same order as `ordered_cbgs`.
    The transition probability from CBG i to CBG j is
    P(i, j) = count(trips between i and j) / count(all trips from i to a CBG).
    :param comb_counts: Total count of all trips between two CGBs.
    :param trip_counts: Total count of all trips from each CGB.
    :returns: Adjacency list with probabilites.
    """
    adjacency_list: Dict[str, List[float]] = {}
        
    for i in ordered_cbgs:
        adjacency_list[i] = []
        for j in ordered_cbgs:

            # count of trips between 
            comb = (i, j)
            trips_between = 0 if not comb in comb_counts else comb_counts[comb]

            # ratio of all trips from i
            p = 0 if not i in trip_counts else trips_between / trip_counts[i]
            adjacency_list[i].append(p)
                
    return adjacency_list

def cum_prob_from_adj_list(adjacency_list: Dict[str, List[float]]) -> Dict[str, np.array]:
    """
    From an adjacency list with transition probabilities, calculate the cumulative probabilities.
    :param adjacency_list: Adjacency list with probabilities.
    :returns: Cummulative probabilities for each item in the adjacency list.
    """
    for key in adjacency_list:
        adjacency_list[key] = np.array(adjacency_list[key]).cumsum()

    return adjacency_list

In [6]:
%%time

adj_list_feb = create_adjacency_list(comb_counts_feb, trip_counts_feb)
adj_list_apr = create_adjacency_list(comb_counts_apr, trip_counts_apr)

CPU times: user 596 ms, sys: 11.8 ms, total: 607 ms
Wall time: 607 ms


In [7]:
# sanity check
print(f"Should be ~{1}, is {sum(adj_list_feb[ordered_cbgs[0]])}")
print(f"Should be ~{1}, is {sum(adj_list_apr[ordered_cbgs[0]])}")

Should be ~1, is 0.9999999999999989
Should be ~1, is 0.9999999999999992


In [8]:
%%time

cum_prob_feb = cum_prob_from_adj_list(adj_list_feb)
cum_prob_apr = cum_prob_from_adj_list(adj_list_apr)

CPU times: user 101 ms, sys: 10.1 ms, total: 111 ms
Wall time: 109 ms


In [9]:
# sanity check
print(f"Should be ~{1}, is {cum_prob_feb[ordered_cbgs[0]][-1]}")
print(f"Should be ~{1}, is {cum_prob_feb[ordered_cbgs[0]][-1]}")

Should be ~1, is 0.9999999999999989
Should be ~1, is 0.9999999999999989


Define the distribution generators...

In [10]:
def household_size_distribution(cbg: str) -> int:
    """
    Household size distribution is drawn from normal distribution with mean 
    according to mean household size of CBG.
    :param cbg: CBG of the household.
    :returns: Household size.
    """
    rng = np.random.default_rng()
    mu = demographics[cbg]['household_size']
    # more realistic sd
    sd = mu / 2
    return max(int(rng.normal(mu, sd)), 1)

def contact_distribution(size: int) -> int:
    """
    Number of nodes in a household that are connected to other households
    :param size: Size of the household.
    :returns: Number of connected nodes.
    """
    rng = np.random.default_rng()
    # return max(int(rng.normal(min(size / 2, 2), 2)), 1)
    # come up with something but I think this probably makes most sense for an undistanced network
    return size


def household_contact_distribution(baseline: float, cbg: str, multiplier: bool) -> int:
    """
    Number of connections from a node to another node outside the household.
    :param baseline: baseline value for the exponent of the degree distribution.
    :param cbg: CBG of the current household.
    :param multiplier: reduce exponent by factor.
    :returns: Number of connections to outside the household.
    """
    # todo should be exponential (with cutoff ?)
    rng = np.random.default_rng()
    multiplier = trip_count_change[cbg]
    if reduced:
        exponent = baseline * trip_count_change[cbg]
    else:
        exponent = baseline
    return max(int(rng.exponential(exponent)), 1)

def draw_rewire_distribution(cbg: str, cum_prob: Dict[str, np.array]) -> str:
    """
    For a given CBG draw from the corresponding rewire distribution and return a random CBG to rewire to.
    :param cbg: CBG to be rewired.
    :param cum_prob: The cummulative probability distribution to draw from.
    """
    rng = np.random.default_rng()
    r = rng.random()
    
    # first instance where p >= r
    idx = next(i for i, p in enumerate(cum_prob[cbg]) if p >= r)
    return ordered_cbgs[idx]

Create the network...

(this is based on Dobson 2020, Chapter Physical Distancing)

In [11]:
def create_network(N, cum_prob_rewire: Dict[str, np.array],
                  baseline: float, multiplier: bool=False) -> nx.Graph:
    """
    Create the network based on some specified distributions.
    :param N: Number of individuals.
    :param cum_prob_rewire: The cummulative probability distribution to draw from for rewiring.
    :param baseline: baseline value for the exponent of the degree distribution.
    :param multiplier: reduce degrees by factor.
    :returns: Network.
    """
    
    # initialise variables
    g = nx.Graph()
    rng = np.random.default_rng()
    
    household_id = 1
    households = []
    cbg_id = 1
    
    # total number of nodes in network
    total_node_ct = 0
    cbg_degree_map = {}

    # add the nodes and create the household connections
    for cbg, demographic in demographics.items():

        # target number of nodes for current CBG
        N_cbg = int(demographic['population_prop'] * N)

        # current number of nodes of this CBG
        n = 0
        
        # initialise for later
        cbg_degree_map[cbg] = []

        if cbg_id > 700:
            break
            ...

        print(f"Creating nodes: CBG {cbg_id} of {len(demographics)}", end="\r")

        # add households
        while n < N_cbg:

            # create household network
            size = household_size_distribution(cbg)
            house_net = nx.complete_graph(size)

            # add unique labels and the current cbg
            nx.relabel_nodes(house_net, lambda l: total_node_ct + l, copy=False)

            # add nodes to the main network
            g.add_nodes_from(house_net.nodes, household=household_id, household_size=size, cbg=cbg)
            g.add_edges_from(house_net.edges, household=household_id, household_size=size, cbg=cbg)
            households.append(house_net)

            # update iteration values
            n += size
            household_id += 1
            total_node_ct += size

        cbg_id += 1

    print('', end='\n')
            
    # number of individuals connected to the outside
    # todo this is probably obsolete in my version since all nodes are connected to the outside
    contacts = []
    for house in households:
        size = house.order()
        num_contacts = contact_distribution(size)
        contacts.append(num_contacts)
        
    # create stubs for connections to outside of household
    stubs = []
    for i in range(len(households)):
        
        print(f"Creating stubs: Household {i+1} of {len(households)}", end="\r")
        
        house = households[i]
        
        # again.. some of this might be obsolete by now (see above). Might just take all ...
        nodes = list(house.nodes())[:contacts[i]]
        
        if len(nodes) > 0:
            cbg = g.nodes[nodes[0]]['cbg']
        
        for node in nodes:
            # draw random degree
            degree = household_contact_distribution(baseline, cbg, reduced)
            
            # append `degree` number of copies of the current node
            new_stubs = [node] * degree
            stubs.extend(new_stubs)
            cbg_degree_map[cbg].extend(new_stubs)
        
    # add one more if number of stubs is odd
    if len(stubs) % 2:
        unique_stubs = list(set(stubs))
        j = rng.integers(len(unique_stubs))
        
        stubs.append(unique_stubs[j])
        cbg_degree_map[g.nodes[unique_stubs[j]]['cbg']].append(unique_stubs[j])
        
        
    print('', end='\n')
    
    # connect stubs according with rewire distribution
    for i in range(0, len(stubs), 2):
        
        print(f"Connecting stubs: {i} of {len(stubs)}", end="\r")
        
        while True:
        
            # draw a CBG from the rewire distribution
            cbg = g.nodes[stubs[i]]['cbg']
            target_cbg = draw_rewire_distribution(cbg, cum_prob_rewire)
            
            # make sure the drawn CBG has any available stubs at all
            if len(cbg_degree_map[target_cbg]) == 0:
                continue
                
            # draw a random stub from the required CBG (preserving degree dist.)
            target_node = rng.choice(cbg_degree_map[target_cbg])
            j = stubs.index(target_node)

            # swap nodes
            stubs[i+1], stubs[j] = stubs[j], stubs[i+1]
            break
    
    print('', end='\n')
    
    # Break up intra-household edges resulting from stubs
    while True:
        
        print("Breaking up pairs...", end="\n")    
        
        # iterate by two successive stubs at a time
        swaps = 0
        for i in range(0, len(stubs), 2):
            
            # break up if successive stubs are of same household
            if g.nodes[stubs[i]]['household'] == g.nodes[stubs[i+1]]['household']:
                
                # swap with random other stub
                j = rng.integers(len(stubs))
                stubs[i+1], stubs[j] = stubs[j], stubs[i+1]
                
                swaps += 1
                
        # no swaps, done.
        if swaps == 0:
            break
    
    # connect pairs of stubs
    for i in range(0, len(stubs), 2):
        # label inter-household edges as houshold 0 of size 0
        g.add_edge(stubs[i], stubs[i+1], household=0, household_size=0)

    print('', end='\n')
    
    return (g, [h.order() for h in households])

In [12]:
%%time
baseline = 3
(g_feb, households_feb) = create_network(N=10000, cum_prob_rewire=cum_prob_feb, 
                                         baseline=baseline, multiplier=False)

(g_apr, households_apr) = create_network(N=10000, cum_prob_rewire=cum_prob_apr, 
                                         baseline=baseline, multiplier=True)

Creating nodes: CBG 628 of 628
Creating stubs: Household 4749 of 4749
Connecting stubs: 28614 of 28616
Breaking up pairs...
Breaking up pairs...

Creating nodes: CBG 628 of 628
Creating stubs: Household 4725 of 4725
Connecting stubs: 10946 of 10948
Breaking up pairs...
Breaking up pairs...

CPU times: user 1min 22s, sys: 2.63 s, total: 1min 25s
Wall time: 1min 24s


Inspect the graph...

In [13]:
print("Feb:")
print('Mean household size of {s:.2f} (range {minf}-{maxf})'.format(s=np.mean(households_feb), minf=min(households_feb), maxf=max(households_feb)))
print('Most connected individual has {k} contacts'.format(k=max(dict(g_feb.degree()).values())))

print("\nApril:")
print('Mean household size of {s:.2f} (range {minf}-{maxf})'.format(s=np.mean(households_apr), minf=min(households_apr), maxf=max(households_apr)))
print('Most connected individual has {k} contacts'.format(k=max(dict(g_apr.degree()).values())))

Feb:
Mean household size of 2.15 (range 1-9)
Most connected individual has 30 contacts

April:
Mean household size of 2.17 (range 1-8)
Most connected individual has 14 contacts


Save the graph ...

In [14]:
nx.write_graphml(g_feb,  'data/graphs/feb_N_10000.graphml')
nx.write_graphml(g_apr,  'data/graphs/apr_N_10000.graphml')

# read like this:
# g_feb = nx.read_graphml('data/graphs/feb_N_10000.graphml')
# g_apr = nx.read_graphml('data/graphs/apr_N_10000.graphml')